# Subset ERA5 from Global grid to NWA25 domain and calculate Specific Huimdity & Total Rain Rate

In [ ]:
# slice down the data
import xarray as xr
import os
import cftime
import numpy as np
from glob import glob
import os

def huss_from_dtas_and_sp(dtas, sp):
    """Compute specific humidity from dewpoint and sea level pressure
    Args:
        dtas (xarray.DataArray): dewpoint at 2m
        sp (xarray.DataAray): surface pressure
    """
    huss = xr.apply_ufunc(huss_from_dtas_and_sp_raw, dtas, sp, dask='parallelized',
                        output_dtypes=[dtas.dtype])
    return huss

def huss_from_dtas_and_sp_raw(dtas, sp):
    """
    compute specific humidity from dewpoint and sea level pressure (ufunc)
    Equation taken from: https://confluence.ecmwf.int/pages/viewpage.action?pageId=171411214
    Parameters
    ----------
    dtas : float/np.ndarray
        dew point at 2m
    sp : float/np.ndarray
        surface pressure
    Returns
    -------
    huss : float/np.ndarray
        specific humidity at 2m
    """
    Rdry=287.0597
    Rvap=461.5250
    a1=611.21
    a3=17.502
    a4=32.19
    T0=273.16
    
    # Calculation of E saturation water vapour from Teten's formula
    E=a1**(a3*(dtas-T0)/(dtas-a4))

    # Calculation of saturation specific humidity at 2m qsat  (equal to huss)

    huss=(Rdry/Rvap)*E/(sp-((1-Rdry/Rvap)*E))
   
    return huss


era5_dict = {'ERA5_sea_ice_cover':'siconc',
            'ERA5_10m_u_component_of_wind':'u10',
            'ERA5_sea_surface_temperature':'sst',
            'ERA5_10m_v_component_of_wind':'v10',
            'ERA5_2m_temperature':'t2m',
            'ERA5_surface_solar_radiation_downwards':'ssrd',
            'ERA5_surface_thermal_radiation_downwards':'strd',
            'ERA5_total_rain_rate':'trr',
            'ERA5_mean_sea_level_pressure':'msl',
            'ERA5_2m_specific_humidity':'huss'}

years=range(1995,1996)
#subset
era5dir = "/Volumes/P5/ERA5/"
subdir2="/Volumes/P8/workdir/james/ERA5/nwa25/subset/"
subdir="/home/james/era5/"
for f in era5_dict.keys():
    print(f)
    for y in years:
        print(y)
        if f=='ERA5_total_rain_rate':
            crr = xr.open_dataset(str(era5dir + 'ERA5_convective_rain_rate_' + str(y) + '.nc')).sel(latitude=slice(65,0), longitude=slice(260,340))
            lsrr = xr.open_dataset(str(era5dir + 'ERA5_large_scale_rain_rate_' + str(y) + '.nc')).sel(latitude=slice(65,0), longitude=slice(260,340))
            trr = crr.drop('crr')
            trr['trr'] = crr['crr'] + lsrr['lsrr']
            trr['trr'].attrs = {'units': 'kg m-2 s-1','long_name': 'Total Rainfall Rate'}
            trr.trr.encoding = {k: v for k, v in crr.crr.encoding.items() if k in {'_FillValue', 'missing_value', 'dtype'}}
            #trr.trr.encoding.update({'add_offset': None, 'scale_factor': None})
            trr.to_netcdf(str(subdir + f + '_' + str(y) + ".nc"), mode='w', format='NETCDF4_CLASSIC')
            crr.close()
            lsrr.close()
            trr.close()
        if f=='ERA5_2m_specific_humidity':
            dtas = xr.open_dataset(str(era5dir + 'ERA5_2m_dewpoint_temperature_' + str(y) + '.nc'))['d2m'].sel(latitude=slice(65,0), longitude=slice(260,340))
            sp = xr.open_dataset(str(era5dir + 'ERA5_surface_pressure_' + str(y) + '.nc'))['sp'].sel(latitude=slice(65,0), longitude=slice(260,340))
            huss = huss_from_dtas_and_sp(dtas, sp)
            huss = huss.to_dataset(name='huss')
            huss.huss.attrs['units'] = 'kg/kg'
            huss.huss.attrs['_FillValue'] = 1e20
            huss.to_netcdf(str(subdir + f + '_' + str(y) + ".nc"),format="NETCDF4_CLASSIC")
            huss.close()
            sp.close()
            dtas.close()
        if 'total_rain_rate' not in f and 'specific_humidity' not in f:
            ds=xr.open_dataset(str(era5dir + f + '_' + str(y) + ".nc")).sel(latitude=slice(65,0), longitude=slice(260,340))
            ds.to_netcdf(str(subdir + f + '_' + str(y) + ".nc"),format="NETCDF4_CLASSIC")
            ds.close()
